# Functionality 2

First, we define shortest_path function, using the Dijkstra’s Algorithm: 

In [1]:
import trees as t
import math
from tqdm import tqdm
import numpy as np
import random
import networkx as nx

In [2]:
def shortest_path(graph, s, t):
    
    if s == t:
        return "There is no path, source and target nodes are the same", -1
    
    unvisited, shortest_path, predecessor = list(), dict(), dict()
    
    # We set the distances between the source node and all other nodes to infinity, except for the distance between source 
    # and itself, which we set to 0.
    for node in graph.nodes():
        shortest_path[node] = math.inf
        unvisited.append(node)
    shortest_path[s] = 0
    
    # We loop untile we visit all the nodes in the graph
    while unvisited:
        # We choose the node with the smallest value as the “current node”
        current_node = None
        for node in unvisited: 
            if current_node == None:
                current_node = node
            elif shortest_path[node] < shortest_path[current_node]:
                current_node = node
        # visit all the  neighbour of current_node. As we visit each neighbor, we update their tentative distance 
        # from the starting node
        for neighbor in graph.neighbors(current_node):
            value = shortest_path[current_node] + graph[current_node][neighbor]['weight']
            if value < shortest_path[neighbor]:
                shortest_path[neighbor] = value
                predecessor[neighbor] = current_node
        unvisited.remove(current_node)
    
    if t not in predecessor:
        return "Not possible, there is no path between target and source", -1
    # now we have to return the path using predecessor dictionary
    last = t
    path = list([last])
    while last != s:
        path.append(predecessor[last])
        last = predecessor[last]
        
    return path, shortest_path[t]  

### Betweenness centrality
Let $n_{s,t}^{i}$ be the number of shortest paths from $s$ to $t$ that pass through $i$ and let $n_{s,t}$ be the total number of shortest paths from $s$ to $t$. Then the betweenness centrality of vertex $i$ is:

$\displaystyle{b_i = \sum_{s, t} \frac{n_{s,t}^{i}}{n_{s,t}}}$

In [3]:
def betweenness(v, graph):
    num, den = 0, 0
    #for each node we compute the shortest path with all the other nodes in the graph
    for source in tqdm(graph.nodes()):
        for target in graph.nodes():
            if source != target and source != v and target != v:
                path, dist = shortest_path(graph, source, target)
                # if exist a shortest path between the source and the target, we update the denominator
                if dist != -1:
                    den += 1
                    # if the node in input is in the shortest path then we update the numerator
                    if v in path:
                        num += 1
    # we change the value of the denominator if it's equal to 0, to avoid dividing by zero                   
    if den == 0:
        den = 1
    betweenness_centrality = num/den
    return betweenness_centrality          

### Closeness centrality
$C(v)={\frac  {N-1}{\sum _{u}d(u,v)}}$
- N is the number of total nodes in the graph
- $d(u,v)$ is the distance between the nodes u and v

In [4]:
def closeness(v, graph):
    N = len(list(graph.nodes()))
    count = 0
    # for each node in the graph we calculate the distance between this node and the node in input
    for node in tqdm(graph.nodes()):
        path, dist = shortest_path(graph, v, node)
        # if exist a path between the node in input v and "node" then we update the count of the distances 
        if dist != -1:
            count += dist
    if count == 0:
        return "There is no path between the node in input and all other nodes in the graph!"
    
    closeness_centrality = (N - 1) / count
    return closeness_centrality

### Degree Centrality

$D(v)={\frac  {degree(v)}{N-1}}$

In [15]:
def degree_centrality(v, graph):
    N = len(list(graph.nodes()))
    out_degree = 0
    in_degree = 0
    for (i,j) in graph.edges():
        # for each edge in the graph we collect the number of outgoing edges from v
        if i == v:
            out_degree += 1
        # for each edge in the graph we collect the number of ingoing edges in v
        if j == v:
            in_degree += 1
    # the degree of a node is given by the sum of the in-degree and out-degree of the node
    degree = in_degree + out_degree
    degree_centrality = degree / (N-1)
    return degree_centrality

### Page rank
First we create the matrix P using this formula:
<p>$P={\frac{\alpha}{N} * M + (1-\alpha)*P_{rw}}$<p>
<p> - $N$ is the number of nodes in the graph<p>
<p> - $M$ is a matrix $N*N$ with all components equal to 1<p>
<p> - $P_{rw}$ is a matrix $N*N$ in which the components are equal to 0, except for the components $a_{i,j}$ for which exist an edge between the node relative to $i$ row and the node relative to $j$ column, for these components the value is given by the normalized weight of the edge<p>

In [6]:
def create_matrix_P(graph, alpha):
    
    N = len(list(graph.nodes()))
    # to make the graph stochastic
    graph = nx.stochastic_graph(graph, weight='weight')
    
    # initialize P_rw and M matrix
    P_rw = np.zeros([N,N])
    M = np.ones([N,N])
    
    # for each node in the graph, we define and memorize its position number in order to build the P_rw matrix
    pos = dict()
    count_pos = 0
    for node in graph.nodes():
        pos[node] = count_pos
        count_pos += 1
    
    # for each node we see the edges that exit from this node
    for node in pos:
        col = list()
        for (i,j) in graph.edges():
        # for each node we collect the node in which each edge goes
            if node == i:
                col.append(j)
        # then we update the component relative to "node" and "elem" of the P_rw matrix 
        for elem in col:
            P_rw[pos[node]][pos[elem]] = graph[node][elem]["weight"]
            
   # finally we build the matrix P
    P = ((alpha / N) * M) + ((1 - alpha) * P_rw)

    return P, pos    

Then, we compute the pagerank algorithm:
- We start from select at random a starting node;
- Then we initialize the $q_0$ vector, such that all of his components are equal to 0, except for the component relative to the starting node, which is equal to 1;
- for each step $t$ of the algorithm we compute: $q_{t} = q_{t-1} * P$ and we check if the algorithm converges;
- if the algorithm have converged we return the number of iterations needed and the page rank value relative to the node $v$ in input.

In [7]:
def pagerank_score(v, graph, alpha, max_iter, tol):
    
    N = len(list(graph.nodes()))
    P, pos = create_matrix_P(graph, alpha)
    
    # picking the start position at random
    random.seed(123)
    start_pos = random.randint(0, N)
    
    # find the position of the node in input using pos dictionary, we will use this position after when returning the pr
    for key, value in pos.items():
        if key == v:
            input_node_pos = value
    
    # initialize the vector q such that there are all zeros except a one in the start_position
    q_start = np.zeros(N)
    q_start[start_pos] = 1
    
    #keep track of the convergence of the algorithm
    convergence = False
    
    for step in tqdm(range(max_iter)):
        # for every step we compute q_t = q_(t-1) * P
        q_t = np.dot(q_start, P)
        # calculate the error in order to check the convergence
        err = sum([abs(q_t[i] - q_start[i]) for i in range(len(q_t))])
        if err < N * tol:
            print("the algorithm Page rank converges in ", step, "iterations")
            return q_t[input_node_pos]
        # update the vector q_start 
        q_start = q_t
        
    if convergence == False:
        print("The algorithm did not converge for ", max_iter, "number of iterations")
    
    return q_t[input_node_pos]

Let's now define the functionality 2 that takes in input <p>
<p> - A user/node <p>
<p> - An interval of time <p>
<p> - One of the following metrics: Betweeness 1, PageRank, ClosenessCentrality 3, DegreeCentrality <p>
The output is the value of the given metric applied over the complete graph for the given interval of time.

In [8]:
def functionality_2(v, start_interval, end_interval, metric, alpha=None, max_iter=None, tol = None):
    graph = t.build("a", start_interval, end_interval)
    if metric == "Betweeness":
        return betweenness(v, graph)
    elif metric == "PageRank" and alpha != None and max_iter != None and tol != None:
        return pagerank_score(v, graph, alpha, max_iter, tol)
    elif metric == "ClosenessCentrality":
        return closeness(v, graph)
    elif metric == "DegreeCentrality":
        return degree_centrality(v, graph)

In [9]:
print(functionality_2(5730934, "2016-01-01", "2016-01-03", "PageRank", alpha=0.85, max_iter=50, tol = 0.000001))

 44%|████████████████████████████████████                                              | 22/50 [00:04<00:05,  4.74it/s]

the algorithm Page rank converges in  22 iterations
1.6309058051812766e-05


In [16]:
print(functionality_2(5730934, '2016-01-01', '2016-01-07', "DegreeCentrality", alpha=None, max_iter=None))

0.00011669973159061734
